In [ ]:
import sys
import pandas as pd
import sklearn

print("python:", sys.executable)
print("pandas:", pd.__version__)
print("sklearn:", sklearn.__version__)


python: c:\Users\tavi9\Desktop\github\ml-churn-starter\.venv\Scripts\python.exe
pandas: 2.3.3
sklearn: 1.8.0


In [2]:
import pandas as pd

df = pd.read_csv("data/WA_Fn-UseC_-Telco-Customer-Churn.csv")

df.head()


,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [3]:
df.shape, df.columns.tolist()

((7043, 21),
 ['customerID',
  'gender',
  'SeniorCitizen',
  'Partner',
  'Dependents',
  'tenure',
  'PhoneService',
  'MultipleLines',
  'InternetService',
  'OnlineSecurity',
  'OnlineBackup',
  'DeviceProtection',
  'TechSupport',
  'StreamingTV',
  'StreamingMovies',
  'Contract',
  'PaperlessBilling',
  'PaymentMethod',
  'MonthlyCharges',
  'TotalCharges',
  'Churn'])

In [4]:
df["Churn"].value_counts()

Churn
No     5174
Yes    1869
Name: count, dtype: int64

In [5]:
x = df.drop(columns=["Churn", "customerID"])
y = df["Churn"]

In [6]:
x.shape, y.shape

((7043, 19), (7043,))

In [7]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(
    x, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

x_train.shape, x_test.shape, y_train.shape, y_test.shape


((5634, 19), (1409, 19), (5634,), (1409,))

In [8]:
x_train.shape, x_test.shape, y_train.shape, y_test.shape


((5634, 19), (1409, 19), (5634,), (1409,))

In [9]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline

# separam coloanele după tip
num_cols = x.select_dtypes(include=["number"]).columns
cat_cols = x.select_dtypes(exclude=["number"]).columns

numeric_transformer = ColumnTransformer(
    transformers=[
        ("num", 
         Pipeline(steps=[
             ("imputer", SimpleImputer(strategy="median")),
             ("scaler", StandardScaler())
         ]), num_cols)
    ],
    remainder="drop"
)

categorical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore"))
])

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, num_cols),
        ("cat", categorical_transformer, cat_cols)
    ]
)


In [10]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

clf = Pipeline(steps=[
    ("preprocess", preprocessor),
    ("model", LogisticRegression(max_iter=1000))
])

clf.fit(x_train, y_train)

y_pred = clf.predict(x_test)

print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))


[[917 118]
 [172 202]]
              precision    recall  f1-score   support

          No       0.84      0.89      0.86      1035
         Yes       0.63      0.54      0.58       374

    accuracy                           0.79      1409
   macro avg       0.74      0.71      0.72      1409
weighted avg       0.79      0.79      0.79      1409



In [11]:
import pandas as pd

model = clf.named_steps["model"]
feature_names = clf.named_steps["preprocess"].get_feature_names_out()
coefs = model.coef_[0]

coef_df = pd.DataFrame({
    "feature": feature_names,
    "coef": coefs
}).sort_values("coef", ascending=False)

coef_df.head(15)


,feature,coef
1321,cat__TotalCharges_20.15,1.335326
1230,cat__TotalCharges_19.9,1.330952
1328,cat__TotalCharges_20.5,1.311257
1320,cat__TotalCharges_20.1,1.181960
2014,cat__TotalCharges_288.05,1.128285
1224,cat__TotalCharges_19.6,1.123792
4650,cat__TotalCharges_740.3,1.029200
1231,cat__TotalCharges_19.95,1.022892
1334,cat__TotalCharges_20.9,1.022670
3113,cat__TotalCharges_45.7,1.015341


In [12]:
df["TotalCharges"] = pd.to_numeric(df["TotalCharges"], errors="coerce")

df["TotalCharges"].isna().sum(), df["TotalCharges"].dtype

(np.int64(11), dtype('float64'))

In [13]:
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline

# refacem x si y DUPA conversia TotalCharges
x = df.drop(columns=["Churn", "customerID"])
y = df["Churn"]

x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=0.2, random_state=42, stratify=y
)

num_cols = x.select_dtypes(include=["number"]).columns
cat_cols = x.select_dtypes(exclude=["number"]).columns

numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore"))
])

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, num_cols),
        ("cat", categorical_transformer, cat_cols)
    ]
)

num_cols, cat_cols

(Index(['SeniorCitizen', 'tenure', 'MonthlyCharges', 'TotalCharges'], dtype='object'),
 Index(['gender', 'Partner', 'Dependents', 'PhoneService', 'MultipleLines',
        'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection',
        'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract',
        'PaperlessBilling', 'PaymentMethod'],
       dtype='object'))

In [14]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

clf = Pipeline(steps=[
    ("preprocess", preprocessor),
    ("model", LogisticRegression(max_iter=1000))
])

clf.fit(x_train, y_train)

y_pred = clf.predict(x_test)

print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[926 109]
 [165 209]]
              precision    recall  f1-score   support

          No       0.85      0.89      0.87      1035
         Yes       0.66      0.56      0.60       374

    accuracy                           0.81      1409
   macro avg       0.75      0.73      0.74      1409
weighted avg       0.80      0.81      0.80      1409



In [15]:
import pandas as pd

model = clf.named_steps["model"]
feature_names = clf.named_steps["preprocess"].get_feature_names_out()
coefs = model.coef_[0]

coef_df = pd.DataFrame({
    "feature": feature_names,
    "coef": coefs
}).sort_values("coef", ascending=False)

# Top factori care CRESC churn
coef_df.head(10)

,feature,coef
16,cat__InternetService_Fiber optic,0.640026
36,cat__Contract_Month-to-month,0.579304
3,num__TotalCharges,0.516708
35,cat__StreamingMovies_Yes,0.204014
32,cat__StreamingTV_Yes,0.203626
43,cat__PaymentMethod_Electronic check,0.198412
18,cat__OnlineSecurity_No,0.157023
27,cat__TechSupport_No,0.132115
14,cat__MultipleLines_Yes,0.105954
0,num__SeniorCitizen,0.053929


In [16]:
# Top factori care SCAD churn
coef_df.tail(10)


,feature,coef
19,cat__OnlineSecurity_No internet service,-0.301082
17,cat__InternetService_No,-0.301082
31,cat__StreamingTV_No internet service,-0.301082
25,cat__DeviceProtection_No internet service,-0.301082
22,cat__OnlineBackup_No internet service,-0.301082
39,cat__PaperlessBilling_No,-0.343260
2,num__MonthlyCharges,-0.596392
15,cat__InternetService_DSL,-0.652406
38,cat__Contract_Two year,-0.772261
1,num__tenure,-1.241319


In [17]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.pipeline import Pipeline

clf_recall = Pipeline(steps=[
    ("preprocess", preprocessor),
    ("model", LogisticRegression(
        max_iter=1000,
        class_weight="balanced"
    ))
])

clf_recall.fit(x_train, y_train)

y_pred_recall = clf_recall.predict(x_test)

print(confusion_matrix(y_test, y_pred_recall))
print(classification_report(y_test, y_pred_recall))


[[747 288]
 [ 81 293]]
              precision    recall  f1-score   support

          No       0.90      0.72      0.80      1035
         Yes       0.50      0.78      0.61       374

    accuracy                           0.74      1409
   macro avg       0.70      0.75      0.71      1409
weighted avg       0.80      0.74      0.75      1409



In [ ]:
# probabilitatea pentru clasa "Yes"
proba_yes = clf.predict_proba(x_test)[:, 1]

proba_yes[:10], proba_yes.min(), proba_yes.max()

(array([0.04529387, 0.68441126, 0.05872074, 0.40170424, 0.02154772,
        0.60393536, 0.44922498, 0.13005522, 0.00288077, 0.39517899]),
 np.float64(0.0017387470659074764),
 np.float64(0.8554356722033837))

In [20]:
import numpy as np

thresholds = [0.5, 0.4, 0.3]

for t in thresholds:
    y_pred_t = np.where(proba_yes >= t, "Yes", "No")

    print(f"\n=== Threshold = {t} ===")
    print(confusion_matrix(y_test, y_pred_t))
    print(classification_report(y_test, y_pred_t))



=== Threshold = 0.5 ===
[[926 109]
 [165 209]]
              precision    recall  f1-score   support

          No       0.85      0.89      0.87      1035
         Yes       0.66      0.56      0.60       374

    accuracy                           0.81      1409
   macro avg       0.75      0.73      0.74      1409
weighted avg       0.80      0.81      0.80      1409


=== Threshold = 0.4 ===
[[846 189]
 [124 250]]
              precision    recall  f1-score   support

          No       0.87      0.82      0.84      1035
         Yes       0.57      0.67      0.62       374

    accuracy                           0.78      1409
   macro avg       0.72      0.74      0.73      1409
weighted avg       0.79      0.78      0.78      1409


=== Threshold = 0.3 ===
[[774 261]
 [ 92 282]]
              precision    recall  f1-score   support

          No       0.89      0.75      0.81      1035
         Yes       0.52      0.75      0.62       374

    accuracy                          

In [21]:
THRESHOLD = 0.3

def predict_with_threshold(model, X, threshold=THRESHOLD):
    proba_yes = model.predict_proba(X)[:, 1]
    return np.where(proba_yes >= threshold, "Yes", "No")

In [22]:
y_pred_prod = predict_with_threshold(clf, x_test)
print(confusion_matrix(y_test, y_pred_prod))
print(classification_report(y_test, y_pred_prod))

[[774 261]
 [ 92 282]]
              precision    recall  f1-score   support

          No       0.89      0.75      0.81      1035
         Yes       0.52      0.75      0.62       374

    accuracy                           0.75      1409
   macro avg       0.71      0.75      0.71      1409
weighted avg       0.79      0.75      0.76      1409

